<a href="https://colab.research.google.com/github/james-t-little/cnn_diss_repo/blob/main/help.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import random
import PIL
from google.colab import drive
from PIL import Image, ImageOps
from tqdm import tqdm
from collections import Counter
import tensorflow as tf

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [8]:
drive.mount('/content/drive')

path = '/content/drive/MyDrive/dissertation_seg_data'

input_path = os.path.join(path,"input_resize_patches/")

mask_path = os.path.join(path,"mask_resize_patches/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Image size that we are going to use
IMG_SIZE = 512
# Our images are RGB (3 channels)
N_CHANNELS = 3
# Scene Parsing has 150 classes + `not labeled`
N_CLASSES = 4

In [10]:
def parse_image(img_path: str) -> dict:
    print(img_path)
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # mask = tf.io.read_file(mask_path)
    # # The masks contain a class index for each pixels
    # mask = tf.image.decode_png(mask, channels=1)
    # # In scene parsing, "not labeled" = 255
    # # But it will mess up with our N_CLASS = 150
    # # Since 255 means the 255th class
    # # Which doesn't exist
    # mask = tf.where(mask == 255, np.dtype('uint8').type(0), mask)
    # # Note that we have to convert the new value (0)
    # # With the same dtype than the tensor itself

    return {'image': image}

In [13]:
input_path + "*.png"

'/content/drive/MyDrive/dissertation_seg_data/input_resize_patches/*.png'

In [16]:
train_dataset = tf.data.Dataset.list_files(input_path + "*.png")
train_dataset = train_dataset.map(parse_image)

train_dataset

Tensor("args_0:0", shape=(), dtype=string)


<MapDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None)}>

In [ ]:
def normalize(input_image: tf.Tensor, input_mask: tf.Tensor) -> tuple:
    
    input_image = tf.cast(input_image, tf.float32) / 255.0
    
    return input_image, input_mask

def load_image_train(datapoint: dict) -> tuple:
    
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

@tf.function
def load_image_test(datapoint: dict) -> tuple:

    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask